# December 19, 2021

https://adventofcode.com/2021/day/19

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
def pnow():
    print( datetime.datetime.now().isoformat() )

In [3]:
def format_data( text ):
    n = 0
    scanners = []
    scn = []
    for line in text.split("\n")[1:]:
        if line[:3] == "---":
            n += 1
            scanners.append(scn)
            scn = []
        elif not line == "":
            scn.append( [int(x) for x in line.split(",")] )

    scanners.append(scn)
    return scanners

In [4]:
with open("../data/2021/19.txt", "r") as f:
    data_str = f.read()
data = format_data( data_str )

In [5]:
with open("../data/2021/19_test.txt", "r") as f:
    test_str = f.read()
test = format_data(test_str)

In [6]:
small_str = '''--- scanner 0 ---
0,2
4,1
3,3

--- scanner 1 ---
-1,-1
-5,0
-2,1'''

In [7]:
small = format_data(small_str)

# Part 1

In [8]:
def beacon_delta( b1, b2 ):
    return [ x-y for x,y in zip(b1,b2) ]

def set_deltas( s1 ):
    return [ [beacon_delta(s1[i], s1[j]) for j in range(len(s1)) if i!=j] for i in range(len(s1)) ]

def rotate_scanner( scanner, rotation ):
    # A vlaid rotation has determinant 1
    # This means an even number of (adjacent) swaps + negations.
    # preserving +x
    if rotation == 0:
        return scanner
    if rotation == 1:
        return [ [ b[0],  b[2], -b[1]] for b in scanner ]
    if rotation == 2:
        return [ [ b[0], -b[2],  b[1]] for b in scanner ]
    if rotation == 3:
        return [ [ b[0], -b[1], -b[2]] for b in scanner ]
    
    # switching +x with -x
    if rotation == 4:
        return [ [-b[0],  b[2],  b[1]] for b in scanner ]
    if rotation == 5:
        return [ [-b[0], -b[2], -b[1]] for b in scanner ]
    if rotation == 6:
        return [ [-b[0], -b[1],  b[2]] for b in scanner ]
    if rotation == 7:
        return [ [-b[0],  b[1], -b[2]] for b in scanner ]
    
    # switch +x with +y
    if rotation == 8:
        return [ [-b[1],  b[0],  b[2]] for b in scanner ]
    if rotation == 9:
        return [ [ b[1],  b[0], -b[2]] for b in scanner ]
    if rotation == 10:
        return [ [ b[2],  b[0],  b[1]] for b in scanner ]
    if rotation == 11:
        return [ [-b[2],  b[0], -b[1]] for b in scanner ]

    # switch +x with -y
    if rotation == 12:
        return [ [ b[1], -b[0],  b[2]] for b in scanner ]
    if rotation == 13:
        return [ [-b[1], -b[0], -b[2]] for b in scanner ]
    if rotation == 14:
        return [ [-b[2], -b[0],  b[1]] for b in scanner ]
    if rotation == 15:
        return [ [ b[2], -b[0], -b[1]] for b in scanner ]
    
    # switch +x with +z
    if rotation == 16:
        return [ [ b[1],  b[2],  b[0]] for b in scanner ]
    if rotation == 17:
        return [ [-b[1], -b[2],  b[0]] for b in scanner ]
    if rotation == 18:
        return [ [-b[2],  b[1],  b[0]] for b in scanner ]
    if rotation == 19:
        return [ [ b[2], -b[1],  b[0]] for b in scanner ]
    
    # switch +x with -z
    if rotation == 20:
        return [ [-b[1],  b[2], -b[0]] for b in scanner ]
    if rotation == 21:
        return [ [ b[1], -b[2], -b[0]] for b in scanner ]
    if rotation == 22:
        return [ [ b[2],  b[1], -b[0]] for b in scanner ]
    if rotation == 23:
        return [ [-b[2], -b[1], -b[0]] for b in scanner ]
    raise Exception("illegal rotation")

def compare_deltas( delta1, delta2, crit = 12 ):
    crit = crit - 1
    n1 = len(delta1) - crit
    n2 = len(delta2) - crit
    
    i1 = 0

    # if we're going to match a point, we're going to match 12 (crit) points
    # so we don't need to check the last 11 (crit-1) points
    while i1 < len(delta1): #n1
        i2 = 0
        while i2 < len(delta2): #n2
            matches = sum([1 for d in delta1[i1] if d in delta2[i2]])
            if matches >= crit:
                return i1, i2
            i2 += 1
        i1 += 1
    return None

def translate_scanner( s1, i1, s2, i2 ):
    '''translate scanner s2 so that it has the same origin as s1 given that s1[i1] and s2[i2] are the same beacon'''
    offset = [ y-x for x,y in zip(s1[i1], s2[i2]) ]
    return [ [b[0] - offset[0], b[1] - offset[1], b[2] - offset[2]] for b in s2], offset

def compare_all_orientations( s1, s2, crit = 12 ):
    '''check if s1 and s2 overlap for some rotation of s2'''
    d1 = set_deltas(s1)
    for rot in range(24):
        srot = rotate_scanner(s2, rot)
        drot = set_deltas(srot)
        result = compare_deltas(d1, drot)
        if result is not None:
            # found a matching beacon!
            return translate_scanner( s1, result[0], srot, result[1] )
        
    return None

def solve_scanners( scanner_list ):
    scanner_pos = {0: [0,0,0]}
    # solved = list of scanners oriented with 0
    solved = [0]
    # scanners solved in current iteration
    cur_solved = [0]
    # scanners solved in prior iteration
    last_solved = []

    # not solved yet
    unsolved = [i for i in range(1, len(scanner_list))]

    while len(unsolved) > 0:
        print( datetime.datetime.now().isoformat() )
        print("Solved:", len(solved), "of", len(scanner_list))
        last_solved = cur_solved
        cur_solved = []
        for unsolved_id in unsolved:
            if unsolved_id in cur_solved:
                continue
            for anchor_id in last_solved:
                result = compare_all_orientations( scanner_list[anchor_id], scanner_list[unsolved_id])
                if result is not None:
                    # result = new_scanned_data, scanner_offset
                    print("matched", unsolved_id, "against", anchor_id)

                    scanner_list[ unsolved_id ] = result[0]
                    cur_solved.append( unsolved_id )
                    solved.append( unsolved_id )
                    scanner_pos[unsolved_id] = result[1]
                    break
        if len(cur_solved) == 0:
            print("I give up")
            return scanner_list
        else:
            for congrats in cur_solved:
                unsolved.remove( congrats )
    
    return scanner_list, scanner_pos

def get_unique_beacons( scanner_list ):
    beacons = []
    for scanner in scanner_list:
        for beacon in scanner:
            if beacon not in beacons:
                beacons.append(beacon)
    return beacons

def part1( scanners ):
    solved_scanners, scanner_pos = solve_scanners( scanners )
    beacons = get_unique_beacons( solved_scanners )
    return len(beacons)

In [9]:
test = format_data(test_str)
len(test)

5

In [10]:
part1(test)

2025-04-02T10:55:37.922110
Solved: 1 of 5
matched 1 against 0
2025-04-02T10:55:38.205173
Solved: 2 of 5
matched 3 against 1
matched 4 against 1
2025-04-02T10:55:38.384787
Solved: 4 of 5
matched 2 against 4


79

In [11]:
data = format_data(data_str)
print(len(data))
part1(data)

37
2025-04-02T10:55:38.494831
Solved: 1 of 37
matched 2 against 0
matched 24 against 0
matched 27 against 0
2025-04-02T10:55:41.653831
Solved: 4 of 37
matched 1 against 27
matched 4 against 27
matched 18 against 2
matched 26 against 27
matched 31 against 24
2025-04-02T10:55:50.074953
Solved: 9 of 37
matched 7 against 18
matched 11 against 4
matched 12 against 4
matched 15 against 1
matched 23 against 18
matched 32 against 1
matched 36 against 1
2025-04-02T10:56:00.087860
Solved: 16 of 37
matched 3 against 32
matched 8 against 12
matched 10 against 15
matched 16 against 36
matched 28 against 36
matched 30 against 12
2025-04-02T10:56:11.754177
Solved: 22 of 37
matched 6 against 3
matched 14 against 28
matched 19 against 3
matched 20 against 3
matched 29 against 16
matched 33 against 16
2025-04-02T10:56:17.650032
Solved: 28 of 37
matched 9 against 6
matched 22 against 33
matched 25 against 14
matched 34 against 6
matched 35 against 6
2025-04-02T10:56:20.498758
Solved: 33 of 37
matched 13 

449

# Part 2

In [12]:
def manhattan( b1, b2 ):
    return abs(b1[0] - b2[0]) + abs(b1[1] - b2[1]) + abs(b1[2] - b2[2])

def max_manhattan( scanner_pos ):
    mm = 0
    for i in range(len(scanner_pos.keys())):
        for j in range(i+1, len(scanner_pos.keys())):
            man = manhattan( scanner_pos[i], scanner_pos[j] )
            if man > mm:
                mm = man
    return mm

def part2( scanners ):
    solved_scanners, scanner_pos = solve_scanners( scanners )
    beacons = get_unique_beacons( solved_scanners )
    mm = max_manhattan(scanner_pos)
    return mm, solved_scanners, scanner_pos, beacons

In [13]:
test = format_data(test_str)
len(test)

5

In [14]:
output = part2(test)

2025-04-02T10:56:21.550352
Solved: 1 of 5
matched 1 against 0
2025-04-02T10:56:21.828012
Solved: 2 of 5
matched 3 against 1
matched 4 against 1
2025-04-02T10:56:21.999964
Solved: 4 of 5
matched 2 against 4


In [15]:
output[0]

3621

In [16]:
len(output[3])

79

In [17]:
data = format_data(data_str)
len(data)
output = part2(data)

2025-04-02T10:56:22.156538
Solved: 1 of 37
matched 2 against 0
matched 24 against 0
matched 27 against 0
2025-04-02T10:56:25.333665
Solved: 4 of 37
matched 1 against 27
matched 4 against 27
matched 18 against 2
matched 26 against 27
matched 31 against 24
2025-04-02T10:56:33.768457
Solved: 9 of 37
matched 7 against 18
matched 11 against 4
matched 12 against 4
matched 15 against 1
matched 23 against 18
matched 32 against 1
matched 36 against 1
2025-04-02T10:56:43.901627
Solved: 16 of 37
matched 3 against 32
matched 8 against 12
matched 10 against 15
matched 16 against 36
matched 28 against 36
matched 30 against 12
2025-04-02T10:56:55.732206
Solved: 22 of 37
matched 6 against 3
matched 14 against 28
matched 19 against 3
matched 20 against 3
matched 29 against 16
matched 33 against 16
2025-04-02T10:57:01.693939
Solved: 28 of 37
matched 9 against 6
matched 22 against 33
matched 25 against 14
matched 34 against 6
matched 35 against 6
2025-04-02T10:57:04.654982
Solved: 33 of 37
matched 13 aga

In [18]:
output[0]

13128